In [1]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras

# import os
# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
SMALL_DATASET_DIR = '../lab1/notMNIST_small/'
LARGE_DATASET_DIR = '../lab1/notMNIST_large/'

LABEL_MAP = {}
INV_LABEL_MAP = {}

In [3]:
def read(data_dir):
    f_v = 0
    
    X, y = [], []
    
    for f in tqdm(os.listdir(data_dir), desc='Letter'):
        
        if not f.startswith('.'):
            img_dir = os.path.join(data_dir, f)
            
            for img in os.listdir(img_dir):
                img_path = os.path.join(img_dir, img)
                data = cv2.imread(img_path, 0)
                
                if data is None:
                    continue
                    
                X.append(data * 2 / 255 - 1)
                
                if LABEL_MAP.get(f) is None:
                    LABEL_MAP[f] = f_v
                    INV_LABEL_MAP[f_v] = f
                    f_v += 1
                    
                y.append(LABEL_MAP[f])
                
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [4]:
def get_split_data(data_dir, size=(0.7, 0.3), random_state=23):
    X, y = read(data_dir)
    assert abs(np.sum(size) - 1.0) < 0.001

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size[1], random_state=random_state)
    return X_train, y_train, X_test, y_test

In [5]:
X_train, y_train, X_val, y_val = get_split_data(SMALL_DATASET_DIR)
X_test, y_test = read(SMALL_DATASET_DIR)

In [6]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)

In [7]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs', 
                                histogram_freq=0, 
                                batch_size=128,
                                write_graph=True,
                                write_grads=True, 
                                write_images=False, 
                                embeddings_freq=0,
                                embeddings_layer_names=None, 
                                embeddings_metadata=None, 
                                embeddings_data=None,
                                update_freq='epoch'),
    keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  min_delta=0.000001, 
                                  patience=3, 
                                  verbose=2, 
                                  mode='auto',
                                  baseline=None,
                                  restore_best_weights=True),
    #keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=2, verbose=2,
    #                                  mode='auto', min_delta=0.001, cooldown=0, min_lr=0),
]

$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (conv: f=4, s=2, c=8) \rightarrow 11x11x8 \rightarrow flatten \rightarrow 968 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [8]:
lr = 0.01
batch_size = 128
epochs = 20

In [9]:
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()

model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=keras.optimizers.SGD(lr=lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

loss, acc = model.evaluate(X_test, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Learning rate: 0.01
Epochs: 20

Instructions for updating:
Colocations handled automatically by placer.
Train on 13106 samples, validate on 5618 samples
Epoch 1/20
13106/13106 [==============================] - ETA: 27s - loss: 2.3536 - acc: 0.12 - ETA: 16s - loss: 2.3429 - acc: 0.12 - ETA: 13s - loss: 2.3353 - acc: 0.12 - ETA: 11s - loss: 2.3326 - acc: 0.12 - ETA: 9s - loss: 2.3276 - acc: 0.1354 - ETA: 8s - loss: 2.3289 - acc: 0.132 - ETA: 7s - loss: 2.3299 - acc: 0.127 - ETA: 7s - loss: 2.3270 - acc: 0.127 - ETA: 7s - loss: 2.3226 - acc: 0.132 - ETA: 7s - loss: 2.3203 - acc: 0.137 - ETA: 6s - loss: 2.3190 - acc: 0.138 - ETA: 6s - loss: 2.3154 - acc: 0.139 - ETA: 6s - loss: 2.3134 - acc: 0.140 - ETA: 6s - loss: 2.3111 - acc: 0.142 - ETA: 6s - loss: 2.3112 - acc: 0.139 - ETA: 6s - loss: 2.3104 - acc: 0.138 - ETA: 6s - loss: 2.3090 - acc: 0.140 - ETA: 6s - loss: 2.3081 - acc: 0.141 - ETA: 5s - loss: 2.3074 - acc: 0.141 - ETA: 5s - loss: 2.3050 - acc: 0.145 - ETA: 5s - loss: 2.3042 - acc

13106/13106 [==============================] - ETA: 5s - loss: 0.6785 - acc: 0.828 - ETA: 5s - loss: 0.5778 - acc: 0.863 - ETA: 5s - loss: 0.5889 - acc: 0.856 - ETA: 4s - loss: 0.5516 - acc: 0.867 - ETA: 4s - loss: 0.5711 - acc: 0.855 - ETA: 4s - loss: 0.5789 - acc: 0.849 - ETA: 4s - loss: 0.5654 - acc: 0.850 - ETA: 4s - loss: 0.5656 - acc: 0.856 - ETA: 4s - loss: 0.5678 - acc: 0.857 - ETA: 4s - loss: 0.5662 - acc: 0.857 - ETA: 4s - loss: 0.5610 - acc: 0.856 - ETA: 4s - loss: 0.5522 - acc: 0.856 - ETA: 4s - loss: 0.5469 - acc: 0.856 - ETA: 4s - loss: 0.5433 - acc: 0.855 - ETA: 4s - loss: 0.5355 - acc: 0.856 - ETA: 4s - loss: 0.5509 - acc: 0.855 - ETA: 4s - loss: 0.5445 - acc: 0.857 - ETA: 4s - loss: 0.5370 - acc: 0.858 - ETA: 4s - loss: 0.5372 - acc: 0.858 - ETA: 4s - loss: 0.5347 - acc: 0.859 - ETA: 4s - loss: 0.5342 - acc: 0.859 - ETA: 4s - loss: 0.5439 - acc: 0.857 - ETA: 4s - loss: 0.5522 - acc: 0.855 - ETA: 4s - loss: 0.5632 - acc: 0.852 - ETA: 4s - loss: 0.5641 - acc: 0.851 - ETA

13106/13106 [==============================] - ETA: 5s - loss: 0.4423 - acc: 0.867 - ETA: 5s - loss: 0.5294 - acc: 0.863 - ETA: 5s - loss: 0.4929 - acc: 0.859 - ETA: 5s - loss: 0.4755 - acc: 0.869 - ETA: 5s - loss: 0.4654 - acc: 0.867 - ETA: 5s - loss: 0.4656 - acc: 0.865 - ETA: 5s - loss: 0.4699 - acc: 0.867 - ETA: 5s - loss: 0.4662 - acc: 0.870 - ETA: 5s - loss: 0.4526 - acc: 0.874 - ETA: 5s - loss: 0.4444 - acc: 0.877 - ETA: 5s - loss: 0.4472 - acc: 0.876 - ETA: 5s - loss: 0.4562 - acc: 0.873 - ETA: 4s - loss: 0.4531 - acc: 0.872 - ETA: 5s - loss: 0.4447 - acc: 0.875 - ETA: 5s - loss: 0.4488 - acc: 0.875 - ETA: 5s - loss: 0.4427 - acc: 0.875 - ETA: 5s - loss: 0.4509 - acc: 0.872 - ETA: 4s - loss: 0.4520 - acc: 0.873 - ETA: 4s - loss: 0.4479 - acc: 0.874 - ETA: 4s - loss: 0.4552 - acc: 0.871 - ETA: 4s - loss: 0.4575 - acc: 0.871 - ETA: 4s - loss: 0.4628 - acc: 0.870 - ETA: 4s - loss: 0.4620 - acc: 0.871 - ETA: 4s - loss: 0.4591 - acc: 0.872 - ETA: 4s - loss: 0.4631 - acc: 0.871 - ETA

18724/18724 [==============================] - ETA: 4s - loss: 0.7979 - acc: 0.781 - ETA: 3s - loss: 0.3260 - acc: 0.921 - ETA: 3s - loss: 0.2943 - acc: 0.930 - ETA: 3s - loss: 0.2772 - acc: 0.931 - ETA: 3s - loss: 0.2811 - acc: 0.924 - ETA: 3s - loss: 0.2825 - acc: 0.923 - ETA: 3s - loss: 0.2796 - acc: 0.921 - ETA: 3s - loss: 0.2808 - acc: 0.917 - ETA: 3s - loss: 0.2951 - acc: 0.912 - ETA: 3s - loss: 0.3065 - acc: 0.908 - ETA: 3s - loss: 0.3200 - acc: 0.903 - ETA: 3s - loss: 0.3508 - acc: 0.895 - ETA: 3s - loss: 0.3836 - acc: 0.884 - ETA: 3s - loss: 0.3974 - acc: 0.880 - ETA: 3s - loss: 0.3989 - acc: 0.878 - ETA: 3s - loss: 0.4127 - acc: 0.874 - ETA: 3s - loss: 0.4027 - acc: 0.879 - ETA: 3s - loss: 0.3914 - acc: 0.884 - ETA: 3s - loss: 0.3839 - acc: 0.887 - ETA: 2s - loss: 0.3778 - acc: 0.889 - ETA: 2s - loss: 0.3749 - acc: 0.889 - ETA: 2s - loss: 0.3675 - acc: 0.892 - ETA: 2s - loss: 0.3625 - acc: 0.894 - ETA: 2s - loss: 0.3614 - acc: 0.894 - ETA: 2s - loss: 0.3617 - acc: 0.895 - ETA

Vertical:
$$28x28 \rightarrow (conv: f=(5, 3), s=1, c=6) \rightarrow 24x26x6 \rightarrow (conv: f=4, s=2, c=8) \rightarrow 11x12x8 \rightarrow flatten \rightarrow 1056 \rightarrow FC \rightarrow 256 \rightarrow FC \rightarrow 64 \rightarrow softmax$$

In [10]:
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()

model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=keras.optimizers.SGD(lr=lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.01
Epochs: 20

Train on 13106 samples, validate on 5618 samples
Epoch 1/20
13106/13106 [==============================] - ETA: 19s - loss: 2.3126 - acc: 0.21 - ETA: 9s - loss: 2.3248 - acc: 0.1719 - ETA: 7s - loss: 2.3170 - acc: 0.170 - ETA: 6s - loss: 2.3105 - acc: 0.175 - ETA: 6s - loss: 2.3069 - acc: 0.176 - ETA: 5s - loss: 2.3028 - acc: 0.182 - ETA: 5s - loss: 2.3004 - acc: 0.182 - ETA: 5s - loss: 2.2977 - acc: 0.186 - ETA: 5s - loss: 2.2943 - acc: 0.193 - ETA: 5s - loss: 2.2927 - acc: 0.196 - ETA: 5s - loss: 2.2924 - acc: 0.195 - ETA: 4s - loss: 2.2859 - acc: 0.200 - ETA: 4s - loss: 2.2835 - acc: 0.200 - ETA: 4s - loss: 2.2814 - acc: 0.202 - ETA: 4s - loss: 2.2796 - acc: 0.201 - ETA: 4s - loss: 2.2779 - acc: 0.198 - ETA: 4s - loss: 2.2756 - acc: 0.201 - ETA: 4s - loss: 2.2711 - acc: 0.203 - ETA: 4s - loss: 2.2664 - acc: 0.207 - ETA: 3s - loss: 2.2634 - acc: 0.210 - ETA: 3s - loss: 2.2586 - acc: 0.213 - ETA: 3s - loss: 2.2550 - acc: 0.215 - ETA:

13106/13106 [==============================] - ETA: 6s - loss: 0.9212 - acc: 0.718 - ETA: 6s - loss: 0.8332 - acc: 0.742 - ETA: 7s - loss: 0.7589 - acc: 0.778 - ETA: 6s - loss: 0.7157 - acc: 0.793 - ETA: 6s - loss: 0.7385 - acc: 0.792 - ETA: 6s - loss: 0.7298 - acc: 0.794 - ETA: 6s - loss: 0.6749 - acc: 0.810 - ETA: 6s - loss: 0.6538 - acc: 0.816 - ETA: 6s - loss: 0.6480 - acc: 0.819 - ETA: 6s - loss: 0.6244 - acc: 0.826 - ETA: 6s - loss: 0.6081 - acc: 0.829 - ETA: 6s - loss: 0.6091 - acc: 0.829 - ETA: 6s - loss: 0.5951 - acc: 0.832 - ETA: 5s - loss: 0.5965 - acc: 0.831 - ETA: 5s - loss: 0.5866 - acc: 0.836 - ETA: 5s - loss: 0.5885 - acc: 0.835 - ETA: 5s - loss: 0.5862 - acc: 0.836 - ETA: 5s - loss: 0.5826 - acc: 0.837 - ETA: 5s - loss: 0.5839 - acc: 0.837 - ETA: 5s - loss: 0.5865 - acc: 0.836 - ETA: 5s - loss: 0.5906 - acc: 0.836 - ETA: 5s - loss: 0.5878 - acc: 0.839 - ETA: 5s - loss: 0.5880 - acc: 0.837 - ETA: 5s - loss: 0.5777 - acc: 0.840 - ETA: 5s - loss: 0.5770 - acc: 0.841 - ETA

13106/13106 [==============================] - ETA: 7s - loss: 0.4085 - acc: 0.867 - ETA: 7s - loss: 0.5432 - acc: 0.847 - ETA: 7s - loss: 0.5347 - acc: 0.838 - ETA: 7s - loss: 0.5215 - acc: 0.843 - ETA: 7s - loss: 0.4936 - acc: 0.854 - ETA: 7s - loss: 0.4902 - acc: 0.851 - ETA: 7s - loss: 0.4879 - acc: 0.851 - ETA: 6s - loss: 0.4738 - acc: 0.855 - ETA: 6s - loss: 0.4763 - acc: 0.860 - ETA: 6s - loss: 0.4839 - acc: 0.857 - ETA: 6s - loss: 0.4842 - acc: 0.858 - ETA: 6s - loss: 0.4822 - acc: 0.861 - ETA: 6s - loss: 0.4792 - acc: 0.861 - ETA: 6s - loss: 0.4753 - acc: 0.863 - ETA: 6s - loss: 0.4645 - acc: 0.867 - ETA: 6s - loss: 0.4636 - acc: 0.868 - ETA: 6s - loss: 0.4553 - acc: 0.870 - ETA: 6s - loss: 0.4568 - acc: 0.873 - ETA: 6s - loss: 0.4549 - acc: 0.875 - ETA: 6s - loss: 0.4514 - acc: 0.877 - ETA: 6s - loss: 0.4548 - acc: 0.875 - ETA: 5s - loss: 0.4552 - acc: 0.875 - ETA: 5s - loss: 0.4516 - acc: 0.876 - ETA: 5s - loss: 0.4493 - acc: 0.877 - ETA: 5s - loss: 0.4454 - acc: 0.877 - ETA

13106/13106 [==============================] - ETA: 5s - loss: 0.4410 - acc: 0.867 - ETA: 5s - loss: 0.3836 - acc: 0.898 - ETA: 5s - loss: 0.3644 - acc: 0.903 - ETA: 5s - loss: 0.3778 - acc: 0.892 - ETA: 5s - loss: 0.3731 - acc: 0.885 - ETA: 5s - loss: 0.3498 - acc: 0.894 - ETA: 5s - loss: 0.3542 - acc: 0.896 - ETA: 5s - loss: 0.3570 - acc: 0.892 - ETA: 5s - loss: 0.3538 - acc: 0.896 - ETA: 5s - loss: 0.3490 - acc: 0.898 - ETA: 5s - loss: 0.3482 - acc: 0.899 - ETA: 5s - loss: 0.3512 - acc: 0.897 - ETA: 5s - loss: 0.3568 - acc: 0.894 - ETA: 5s - loss: 0.3704 - acc: 0.891 - ETA: 5s - loss: 0.3750 - acc: 0.891 - ETA: 5s - loss: 0.3690 - acc: 0.893 - ETA: 5s - loss: 0.3696 - acc: 0.892 - ETA: 5s - loss: 0.3695 - acc: 0.893 - ETA: 5s - loss: 0.3762 - acc: 0.891 - ETA: 5s - loss: 0.3782 - acc: 0.892 - ETA: 5s - loss: 0.3800 - acc: 0.891 - ETA: 5s - loss: 0.3783 - acc: 0.892 - ETA: 5s - loss: 0.3799 - acc: 0.893 - ETA: 5s - loss: 0.3743 - acc: 0.894 - ETA: 5s - loss: 0.3790 - acc: 0.894 - ETA

13106/13106 [==============================] - ETA: 5s - loss: 0.4118 - acc: 0.875 - ETA: 5s - loss: 0.3449 - acc: 0.902 - ETA: 5s - loss: 0.3820 - acc: 0.893 - ETA: 5s - loss: 0.4073 - acc: 0.888 - ETA: 5s - loss: 0.3724 - acc: 0.896 - ETA: 5s - loss: 0.3718 - acc: 0.895 - ETA: 5s - loss: 0.3695 - acc: 0.892 - ETA: 5s - loss: 0.3609 - acc: 0.896 - ETA: 5s - loss: 0.3566 - acc: 0.897 - ETA: 5s - loss: 0.3437 - acc: 0.903 - ETA: 5s - loss: 0.3432 - acc: 0.901 - ETA: 5s - loss: 0.3386 - acc: 0.904 - ETA: 5s - loss: 0.3367 - acc: 0.903 - ETA: 5s - loss: 0.3388 - acc: 0.905 - ETA: 5s - loss: 0.3315 - acc: 0.908 - ETA: 5s - loss: 0.3241 - acc: 0.911 - ETA: 5s - loss: 0.3285 - acc: 0.911 - ETA: 5s - loss: 0.3284 - acc: 0.911 - ETA: 5s - loss: 0.3312 - acc: 0.908 - ETA: 5s - loss: 0.3332 - acc: 0.908 - ETA: 5s - loss: 0.3277 - acc: 0.910 - ETA: 5s - loss: 0.3261 - acc: 0.910 - ETA: 4s - loss: 0.3282 - acc: 0.910 - ETA: 4s - loss: 0.3319 - acc: 0.908 - ETA: 4s - loss: 0.3331 - acc: 0.907 - ETA

LeNet-5 + vertical:
$$
28x28 \rightarrow (conv: f=(5, 3), s=1, c=6, tanh) \rightarrow 24x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x13x6 \rightarrow (conv: f=3, s=1, c=8, tanh) \rightarrow 10x11x8 \rightarrow (maxpool: f=2, s=2) \rightarrow 5x5x8 \rightarrow flatten \rightarrow 200 \rightarrow FC(relu) \rightarrow 120 \rightarrow FC(relu) \rightarrow 84 \rightarrow softmax
$$

In [11]:
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()

model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(84, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=keras.optimizers.SGD(lr=lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.01
Epochs: 20

Train on 13106 samples, validate on 5618 samples
Epoch 1/20
13106/13106 [==============================] - ETA: 20s - loss: 2.2914 - acc: 0.13 - ETA: 12s - loss: 2.3023 - acc: 0.12 - ETA: 10s - loss: 2.3012 - acc: 0.12 - ETA: 9s - loss: 2.3058 - acc: 0.1133 - ETA: 8s - loss: 2.3039 - acc: 0.103 - ETA: 7s - loss: 2.3062 - acc: 0.096 - ETA: 7s - loss: 2.3084 - acc: 0.088 - ETA: 7s - loss: 2.3089 - acc: 0.088 - ETA: 6s - loss: 2.3086 - acc: 0.090 - ETA: 6s - loss: 2.3071 - acc: 0.091 - ETA: 6s - loss: 2.3055 - acc: 0.095 - ETA: 6s - loss: 2.3053 - acc: 0.095 - ETA: 6s - loss: 2.3034 - acc: 0.099 - ETA: 6s - loss: 2.3020 - acc: 0.099 - ETA: 5s - loss: 2.3005 - acc: 0.101 - ETA: 5s - loss: 2.3000 - acc: 0.102 - ETA: 5s - loss: 2.2989 - acc: 0.104 - ETA: 5s - loss: 2.2992 - acc: 0.104 - ETA: 4s - loss: 2.2991 - acc: 0.104 - ETA: 4s - loss: 2.2979 - acc: 0.105 - ETA: 4s - loss: 2.2966 - acc: 0.108 - ETA: 4s - loss: 2.2955 - acc: 0.110 - ETA:

13106/13106 [==============================] - ETA: 5s - loss: 1.1773 - acc: 0.734 - ETA: 5s - loss: 1.1904 - acc: 0.710 - ETA: 5s - loss: 1.2283 - acc: 0.690 - ETA: 5s - loss: 1.2048 - acc: 0.703 - ETA: 5s - loss: 1.1938 - acc: 0.706 - ETA: 5s - loss: 1.1995 - acc: 0.705 - ETA: 5s - loss: 1.1799 - acc: 0.712 - ETA: 5s - loss: 1.1744 - acc: 0.717 - ETA: 5s - loss: 1.1706 - acc: 0.720 - ETA: 4s - loss: 1.1714 - acc: 0.715 - ETA: 5s - loss: 1.1736 - acc: 0.714 - ETA: 4s - loss: 1.1706 - acc: 0.716 - ETA: 4s - loss: 1.1667 - acc: 0.718 - ETA: 4s - loss: 1.1626 - acc: 0.719 - ETA: 4s - loss: 1.1535 - acc: 0.722 - ETA: 4s - loss: 1.1419 - acc: 0.727 - ETA: 4s - loss: 1.1355 - acc: 0.728 - ETA: 4s - loss: 1.1377 - acc: 0.726 - ETA: 4s - loss: 1.1361 - acc: 0.725 - ETA: 3s - loss: 1.1358 - acc: 0.725 - ETA: 3s - loss: 1.1321 - acc: 0.724 - ETA: 3s - loss: 1.1284 - acc: 0.724 - ETA: 3s - loss: 1.1300 - acc: 0.721 - ETA: 3s - loss: 1.1307 - acc: 0.720 - ETA: 3s - loss: 1.1296 - acc: 0.721 - ETA

13106/13106 [==============================] - ETA: 3s - loss: 0.6574 - acc: 0.828 - ETA: 3s - loss: 0.6349 - acc: 0.817 - ETA: 3s - loss: 0.6511 - acc: 0.814 - ETA: 3s - loss: 0.6516 - acc: 0.821 - ETA: 3s - loss: 0.6520 - acc: 0.830 - ETA: 3s - loss: 0.6562 - acc: 0.828 - ETA: 3s - loss: 0.6548 - acc: 0.826 - ETA: 3s - loss: 0.6595 - acc: 0.826 - ETA: 3s - loss: 0.6555 - acc: 0.829 - ETA: 3s - loss: 0.6489 - acc: 0.831 - ETA: 3s - loss: 0.6547 - acc: 0.828 - ETA: 3s - loss: 0.6578 - acc: 0.825 - ETA: 3s - loss: 0.6602 - acc: 0.825 - ETA: 3s - loss: 0.6620 - acc: 0.824 - ETA: 2s - loss: 0.6565 - acc: 0.824 - ETA: 2s - loss: 0.6518 - acc: 0.825 - ETA: 2s - loss: 0.6520 - acc: 0.825 - ETA: 2s - loss: 0.6477 - acc: 0.826 - ETA: 2s - loss: 0.6503 - acc: 0.825 - ETA: 2s - loss: 0.6526 - acc: 0.824 - ETA: 2s - loss: 0.6517 - acc: 0.825 - ETA: 2s - loss: 0.6543 - acc: 0.825 - ETA: 2s - loss: 0.6500 - acc: 0.826 - ETA: 2s - loss: 0.6458 - acc: 0.827 - ETA: 2s - loss: 0.6454 - acc: 0.826 - ETA

13106/13106 [==============================] - ETA: 5s - loss: 0.5946 - acc: 0.851 - ETA: 5s - loss: 0.6186 - acc: 0.843 - ETA: 5s - loss: 0.5882 - acc: 0.841 - ETA: 5s - loss: 0.6039 - acc: 0.841 - ETA: 5s - loss: 0.5816 - acc: 0.845 - ETA: 5s - loss: 0.5826 - acc: 0.846 - ETA: 5s - loss: 0.5645 - acc: 0.849 - ETA: 5s - loss: 0.5533 - acc: 0.850 - ETA: 5s - loss: 0.5696 - acc: 0.845 - ETA: 4s - loss: 0.5650 - acc: 0.849 - ETA: 4s - loss: 0.5569 - acc: 0.849 - ETA: 4s - loss: 0.5515 - acc: 0.849 - ETA: 4s - loss: 0.5293 - acc: 0.857 - ETA: 4s - loss: 0.5364 - acc: 0.855 - ETA: 4s - loss: 0.5324 - acc: 0.856 - ETA: 4s - loss: 0.5394 - acc: 0.854 - ETA: 4s - loss: 0.5418 - acc: 0.853 - ETA: 4s - loss: 0.5369 - acc: 0.856 - ETA: 3s - loss: 0.5370 - acc: 0.856 - ETA: 3s - loss: 0.5364 - acc: 0.855 - ETA: 3s - loss: 0.5386 - acc: 0.855 - ETA: 3s - loss: 0.5346 - acc: 0.856 - ETA: 3s - loss: 0.5340 - acc: 0.856 - ETA: 3s - loss: 0.5303 - acc: 0.857 - ETA: 3s - loss: 0.5303 - acc: 0.857 - ETA

13106/13106 [==============================] - ETA: 3s - loss: 0.4200 - acc: 0.859 - ETA: 3s - loss: 0.4588 - acc: 0.869 - ETA: 3s - loss: 0.4568 - acc: 0.871 - ETA: 3s - loss: 0.4693 - acc: 0.865 - ETA: 3s - loss: 0.4780 - acc: 0.865 - ETA: 3s - loss: 0.4829 - acc: 0.864 - ETA: 3s - loss: 0.4958 - acc: 0.859 - ETA: 3s - loss: 0.4920 - acc: 0.863 - ETA: 3s - loss: 0.4820 - acc: 0.868 - ETA: 3s - loss: 0.4855 - acc: 0.868 - ETA: 3s - loss: 0.4867 - acc: 0.868 - ETA: 3s - loss: 0.4921 - acc: 0.865 - ETA: 3s - loss: 0.4940 - acc: 0.865 - ETA: 3s - loss: 0.4904 - acc: 0.866 - ETA: 2s - loss: 0.4949 - acc: 0.865 - ETA: 2s - loss: 0.4902 - acc: 0.866 - ETA: 2s - loss: 0.4910 - acc: 0.865 - ETA: 2s - loss: 0.4846 - acc: 0.867 - ETA: 2s - loss: 0.4876 - acc: 0.865 - ETA: 2s - loss: 0.4914 - acc: 0.865 - ETA: 2s - loss: 0.4913 - acc: 0.864 - ETA: 2s - loss: 0.4912 - acc: 0.864 - ETA: 2s - loss: 0.4907 - acc: 0.864 - ETA: 2s - loss: 0.4884 - acc: 0.865 - ETA: 2s - loss: 0.4891 - acc: 0.865 - ETA

Custom with maxpool:
$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 13x13x6 \rightarrow (conv: f=3, s=2, c=16) \rightarrow 5x5x16 \rightarrow (maxpool: f=2, s=1) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 258 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [12]:
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()

model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=(2, 2), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=keras.optimizers.SGD(lr=lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.01
Epochs: 20

Train on 13106 samples, validate on 5618 samples
Epoch 1/20
13106/13106 [==============================] - ETA: 17s - loss: 2.4395 - acc: 0.04 - ETA: 7s - loss: 2.3655 - acc: 0.1068 - ETA: 5s - loss: 2.3518 - acc: 0.110 - ETA: 4s - loss: 2.3486 - acc: 0.116 - ETA: 4s - loss: 2.3455 - acc: 0.117 - ETA: 3s - loss: 2.3386 - acc: 0.117 - ETA: 3s - loss: 2.3288 - acc: 0.114 - ETA: 3s - loss: 2.3235 - acc: 0.114 - ETA: 3s - loss: 2.3203 - acc: 0.115 - ETA: 3s - loss: 2.3192 - acc: 0.116 - ETA: 3s - loss: 2.3136 - acc: 0.120 - ETA: 3s - loss: 2.3112 - acc: 0.119 - ETA: 3s - loss: 2.3103 - acc: 0.118 - ETA: 3s - loss: 2.3076 - acc: 0.118 - ETA: 3s - loss: 2.3059 - acc: 0.118 - ETA: 2s - loss: 2.3043 - acc: 0.120 - ETA: 2s - loss: 2.3014 - acc: 0.124 - ETA: 2s - loss: 2.2988 - acc: 0.125 - ETA: 2s - loss: 2.2968 - acc: 0.127 - ETA: 2s - loss: 2.2961 - acc: 0.128 - ETA: 2s - loss: 2.2954 - acc: 0.128 - ETA: 2s - loss: 2.2953 - acc: 0.127 - ETA:

13106/13106 [==============================] - ETA: 4s - loss: 0.7744 - acc: 0.804 - ETA: 3s - loss: 0.7325 - acc: 0.812 - ETA: 3s - loss: 0.7265 - acc: 0.803 - ETA: 4s - loss: 0.7326 - acc: 0.800 - ETA: 4s - loss: 0.7272 - acc: 0.802 - ETA: 4s - loss: 0.7248 - acc: 0.800 - ETA: 4s - loss: 0.7186 - acc: 0.802 - ETA: 3s - loss: 0.7316 - acc: 0.794 - ETA: 3s - loss: 0.7106 - acc: 0.799 - ETA: 3s - loss: 0.7185 - acc: 0.798 - ETA: 3s - loss: 0.7128 - acc: 0.801 - ETA: 3s - loss: 0.7145 - acc: 0.803 - ETA: 3s - loss: 0.7121 - acc: 0.804 - ETA: 3s - loss: 0.7011 - acc: 0.810 - ETA: 3s - loss: 0.6974 - acc: 0.811 - ETA: 3s - loss: 0.7005 - acc: 0.810 - ETA: 2s - loss: 0.6968 - acc: 0.812 - ETA: 2s - loss: 0.6994 - acc: 0.812 - ETA: 2s - loss: 0.6983 - acc: 0.811 - ETA: 2s - loss: 0.6995 - acc: 0.810 - ETA: 2s - loss: 0.6976 - acc: 0.811 - ETA: 2s - loss: 0.6967 - acc: 0.811 - ETA: 2s - loss: 0.7003 - acc: 0.811 - ETA: 2s - loss: 0.6976 - acc: 0.810 - ETA: 2s - loss: 0.6948 - acc: 0.810 - ETA

13106/13106 [==============================] - ETA: 3s - loss: 0.5424 - acc: 0.867 - ETA: 3s - loss: 0.5285 - acc: 0.867 - ETA: 3s - loss: 0.4872 - acc: 0.871 - ETA: 3s - loss: 0.4820 - acc: 0.873 - ETA: 3s - loss: 0.4925 - acc: 0.871 - ETA: 3s - loss: 0.4868 - acc: 0.872 - ETA: 3s - loss: 0.4910 - acc: 0.870 - ETA: 2s - loss: 0.4653 - acc: 0.877 - ETA: 2s - loss: 0.4589 - acc: 0.877 - ETA: 2s - loss: 0.4677 - acc: 0.875 - ETA: 2s - loss: 0.4613 - acc: 0.875 - ETA: 2s - loss: 0.4705 - acc: 0.871 - ETA: 2s - loss: 0.4722 - acc: 0.871 - ETA: 2s - loss: 0.4729 - acc: 0.870 - ETA: 2s - loss: 0.4724 - acc: 0.870 - ETA: 2s - loss: 0.4728 - acc: 0.870 - ETA: 2s - loss: 0.4707 - acc: 0.871 - ETA: 2s - loss: 0.4663 - acc: 0.873 - ETA: 2s - loss: 0.4603 - acc: 0.874 - ETA: 2s - loss: 0.4590 - acc: 0.874 - ETA: 2s - loss: 0.4556 - acc: 0.875 - ETA: 1s - loss: 0.4564 - acc: 0.875 - ETA: 1s - loss: 0.4575 - acc: 0.875 - ETA: 1s - loss: 0.4563 - acc: 0.875 - ETA: 1s - loss: 0.4566 - acc: 0.876 - ETA

13106/13106 [==============================] - ETA: 3s - loss: 0.5386 - acc: 0.843 - ETA: 7s - loss: 0.4507 - acc: 0.882 - ETA: 6s - loss: 0.3880 - acc: 0.896 - ETA: 6s - loss: 0.3948 - acc: 0.893 - ETA: 6s - loss: 0.3753 - acc: 0.899 - ETA: 6s - loss: 0.3828 - acc: 0.896 - ETA: 6s - loss: 0.3987 - acc: 0.889 - ETA: 6s - loss: 0.3887 - acc: 0.892 - ETA: 6s - loss: 0.3879 - acc: 0.892 - ETA: 6s - loss: 0.3932 - acc: 0.892 - ETA: 5s - loss: 0.3808 - acc: 0.893 - ETA: 5s - loss: 0.3779 - acc: 0.893 - ETA: 5s - loss: 0.3785 - acc: 0.892 - ETA: 5s - loss: 0.3796 - acc: 0.891 - ETA: 5s - loss: 0.3773 - acc: 0.891 - ETA: 4s - loss: 0.3840 - acc: 0.891 - ETA: 4s - loss: 0.3827 - acc: 0.891 - ETA: 4s - loss: 0.3824 - acc: 0.891 - ETA: 4s - loss: 0.3777 - acc: 0.892 - ETA: 4s - loss: 0.3814 - acc: 0.890 - ETA: 4s - loss: 0.3829 - acc: 0.889 - ETA: 4s - loss: 0.3825 - acc: 0.889 - ETA: 4s - loss: 0.3815 - acc: 0.889 - ETA: 4s - loss: 0.3766 - acc: 0.890 - ETA: 4s - loss: 0.3805 - acc: 0.890 - ETA

LeNet-5:
$$
28x28 \rightarrow (conv: f=5, s=1, c=6, tanh) \rightarrow 24x24x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x12x6 \rightarrow (conv: f=5, s=1, c=16) \rightarrow 8x8x16 \rightarrow (maxpool: f=2, s=2) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 256 \rightarrow FC \rightarrow 128 \rightarrow FC \rightarrow 64 \rightarrow softmax
$$

In [13]:
print('Building LeNet-5...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()


model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='tanh'),
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=keras.optimizers.SGD(lr=lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building LeNet-5...
Learning rate: 0.01
Epochs: 20

Train on 13106 samples, validate on 5618 samples
Epoch 1/20
13106/13106 [==============================] - ETA: 2:06 - loss: 2.3890 - acc: 0.046 - ETA: 1:08 - loss: 2.3785 - acc: 0.074 - ETA: 48s - loss: 2.3724 - acc: 0.072 - ETA: 38s - loss: 2.3733 - acc: 0.07 - ETA: 32s - loss: 2.3725 - acc: 0.06 - ETA: 29s - loss: 2.3651 - acc: 0.07 - ETA: 26s - loss: 2.3587 - acc: 0.07 - ETA: 23s - loss: 2.3526 - acc: 0.07 - ETA: 21s - loss: 2.3465 - acc: 0.06 - ETA: 20s - loss: 2.3417 - acc: 0.06 - ETA: 18s - loss: 2.3375 - acc: 0.06 - ETA: 18s - loss: 2.3340 - acc: 0.06 - ETA: 17s - loss: 2.3314 - acc: 0.06 - ETA: 16s - loss: 2.3288 - acc: 0.06 - ETA: 15s - loss: 2.3247 - acc: 0.07 - ETA: 15s - loss: 2.3228 - acc: 0.07 - ETA: 14s - loss: 2.3197 - acc: 0.07 - ETA: 14s - loss: 2.3163 - acc: 0.07 - ETA: 13s - loss: 2.3134 - acc: 0.07 - ETA: 13s - loss: 2.3106 - acc: 0.07 - ETA: 12s - loss: 2.3078 - acc: 0.07 - ETA: 12s - loss: 2.3051 - acc: 0.08 - 

13106/13106 [==============================] - ETA: 6s - loss: 0.7856 - acc: 0.843 - ETA: 6s - loss: 0.7124 - acc: 0.855 - ETA: 7s - loss: 0.7388 - acc: 0.851 - ETA: 7s - loss: 0.7402 - acc: 0.853 - ETA: 7s - loss: 0.7800 - acc: 0.832 - ETA: 7s - loss: 0.7854 - acc: 0.822 - ETA: 7s - loss: 0.7911 - acc: 0.821 - ETA: 7s - loss: 0.7967 - acc: 0.816 - ETA: 7s - loss: 0.8095 - acc: 0.811 - ETA: 7s - loss: 0.8016 - acc: 0.814 - ETA: 7s - loss: 0.8191 - acc: 0.807 - ETA: 7s - loss: 0.8216 - acc: 0.804 - ETA: 6s - loss: 0.8208 - acc: 0.803 - ETA: 7s - loss: 0.8160 - acc: 0.804 - ETA: 6s - loss: 0.8119 - acc: 0.807 - ETA: 6s - loss: 0.8104 - acc: 0.808 - ETA: 6s - loss: 0.8091 - acc: 0.808 - ETA: 6s - loss: 0.8113 - acc: 0.807 - ETA: 6s - loss: 0.8108 - acc: 0.808 - ETA: 6s - loss: 0.8172 - acc: 0.806 - ETA: 6s - loss: 0.8147 - acc: 0.807 - ETA: 6s - loss: 0.8122 - acc: 0.808 - ETA: 6s - loss: 0.8101 - acc: 0.810 - ETA: 6s - loss: 0.8084 - acc: 0.809 - ETA: 6s - loss: 0.8072 - acc: 0.809 - ETA

13106/13106 [==============================] - ETA: 9s - loss: 0.6505 - acc: 0.835 - ETA: 9s - loss: 0.5740 - acc: 0.851 - ETA: 9s - loss: 0.5572 - acc: 0.854 - ETA: 9s - loss: 0.5492 - acc: 0.859 - ETA: 9s - loss: 0.5385 - acc: 0.857 - ETA: 9s - loss: 0.5289 - acc: 0.864 - ETA: 9s - loss: 0.5225 - acc: 0.868 - ETA: 8s - loss: 0.5567 - acc: 0.860 - ETA: 8s - loss: 0.5454 - acc: 0.862 - ETA: 8s - loss: 0.5422 - acc: 0.861 - ETA: 8s - loss: 0.5390 - acc: 0.863 - ETA: 8s - loss: 0.5475 - acc: 0.861 - ETA: 8s - loss: 0.5621 - acc: 0.858 - ETA: 8s - loss: 0.5588 - acc: 0.859 - ETA: 8s - loss: 0.5607 - acc: 0.858 - ETA: 8s - loss: 0.5684 - acc: 0.856 - ETA: 8s - loss: 0.5679 - acc: 0.857 - ETA: 8s - loss: 0.5808 - acc: 0.852 - ETA: 8s - loss: 0.5813 - acc: 0.851 - ETA: 8s - loss: 0.5802 - acc: 0.849 - ETA: 7s - loss: 0.5751 - acc: 0.851 - ETA: 7s - loss: 0.5816 - acc: 0.850 - ETA: 7s - loss: 0.5784 - acc: 0.851 - ETA: 7s - loss: 0.5769 - acc: 0.852 - ETA: 7s - loss: 0.5752 - acc: 0.852 - ETA

13106/13106 [==============================] - ETA: 5s - loss: 0.4993 - acc: 0.882 - ETA: 6s - loss: 0.5432 - acc: 0.867 - ETA: 6s - loss: 0.5446 - acc: 0.867 - ETA: 6s - loss: 0.5603 - acc: 0.857 - ETA: 7s - loss: 0.5420 - acc: 0.859 - ETA: 7s - loss: 0.5525 - acc: 0.859 - ETA: 7s - loss: 0.5338 - acc: 0.862 - ETA: 7s - loss: 0.5503 - acc: 0.858 - ETA: 7s - loss: 0.5506 - acc: 0.856 - ETA: 7s - loss: 0.5494 - acc: 0.856 - ETA: 6s - loss: 0.5477 - acc: 0.854 - ETA: 6s - loss: 0.5485 - acc: 0.854 - ETA: 6s - loss: 0.5515 - acc: 0.853 - ETA: 6s - loss: 0.5481 - acc: 0.852 - ETA: 6s - loss: 0.5361 - acc: 0.856 - ETA: 6s - loss: 0.5365 - acc: 0.855 - ETA: 6s - loss: 0.5331 - acc: 0.856 - ETA: 6s - loss: 0.5239 - acc: 0.858 - ETA: 6s - loss: 0.5174 - acc: 0.861 - ETA: 5s - loss: 0.5134 - acc: 0.862 - ETA: 5s - loss: 0.5100 - acc: 0.863 - ETA: 5s - loss: 0.5109 - acc: 0.864 - ETA: 5s - loss: 0.5064 - acc: 0.865 - ETA: 5s - loss: 0.5062 - acc: 0.865 - ETA: 5s - loss: 0.5059 - acc: 0.867 - ETA

13106/13106 [==============================] - ETA: 5s - loss: 0.3913 - acc: 0.906 - ETA: 5s - loss: 0.5101 - acc: 0.875 - ETA: 5s - loss: 0.4978 - acc: 0.877 - ETA: 5s - loss: 0.4741 - acc: 0.867 - ETA: 5s - loss: 0.4897 - acc: 0.864 - ETA: 5s - loss: 0.5162 - acc: 0.858 - ETA: 5s - loss: 0.4899 - acc: 0.867 - ETA: 5s - loss: 0.4998 - acc: 0.868 - ETA: 5s - loss: 0.4874 - acc: 0.870 - ETA: 5s - loss: 0.4860 - acc: 0.871 - ETA: 4s - loss: 0.4899 - acc: 0.870 - ETA: 4s - loss: 0.4832 - acc: 0.873 - ETA: 4s - loss: 0.4865 - acc: 0.871 - ETA: 4s - loss: 0.4798 - acc: 0.872 - ETA: 4s - loss: 0.4809 - acc: 0.871 - ETA: 4s - loss: 0.4750 - acc: 0.872 - ETA: 4s - loss: 0.4737 - acc: 0.873 - ETA: 4s - loss: 0.4702 - acc: 0.872 - ETA: 4s - loss: 0.4647 - acc: 0.874 - ETA: 4s - loss: 0.4661 - acc: 0.874 - ETA: 4s - loss: 0.4612 - acc: 0.876 - ETA: 4s - loss: 0.4574 - acc: 0.876 - ETA: 4s - loss: 0.4607 - acc: 0.876 - ETA: 4s - loss: 0.4542 - acc: 0.878 - ETA: 4s - loss: 0.4511 - acc: 0.880 - ETA

18724/18724 [==============================] - ETA: 5s - loss: 1.0163 - acc: 0.750 - ETA: 5s - loss: 0.4852 - acc: 0.852 - ETA: 5s - loss: 0.4131 - acc: 0.872 - ETA: 5s - loss: 0.4471 - acc: 0.863 - ETA: 5s - loss: 0.4279 - acc: 0.862 - ETA: 5s - loss: 0.4444 - acc: 0.854 - ETA: 5s - loss: 0.4329 - acc: 0.860 - ETA: 5s - loss: 0.4417 - acc: 0.858 - ETA: 5s - loss: 0.4384 - acc: 0.855 - ETA: 4s - loss: 0.4390 - acc: 0.856 - ETA: 4s - loss: 0.4530 - acc: 0.852 - ETA: 4s - loss: 0.4504 - acc: 0.856 - ETA: 4s - loss: 0.4472 - acc: 0.856 - ETA: 4s - loss: 0.4466 - acc: 0.858 - ETA: 4s - loss: 0.4649 - acc: 0.854 - ETA: 4s - loss: 0.4749 - acc: 0.851 - ETA: 4s - loss: 0.4797 - acc: 0.850 - ETA: 4s - loss: 0.4738 - acc: 0.853 - ETA: 4s - loss: 0.4785 - acc: 0.853 - ETA: 4s - loss: 0.4723 - acc: 0.855 - ETA: 4s - loss: 0.4738 - acc: 0.855 - ETA: 4s - loss: 0.4803 - acc: 0.852 - ETA: 4s - loss: 0.4677 - acc: 0.858 - ETA: 4s - loss: 0.4600 - acc: 0.861 - ETA: 4s - loss: 0.4552 - acc: 0.864 - ETA